In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers, Model
import matplotlib.pyplot as plt 
import seaborn as sns
import os
import glob
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler

2025-02-09 21:39:03.345142: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F  
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm

In [3]:
'''
        Each file contains nine columns and a different number of rows depending on the test length.
        
        1st column is the acceleration data in the X axis measured by the sensor ADXL345.
        2nd column is the acceleration data in the Y axis measured by the sensor ADXL345.
        3rd column is the acceleration data in the Z axis measured by the sensor ADXL345.

        4th column is the rotation data in the X axis measured by the sensor ITG3200.
        5th column is the rotation data in the Y axis measured by the sensor ITG3200.
        6th column is the rotation data in the Z axis measured by the sensor ITG3200.

        7th column is the acceleration data in the X axis measured by the sensor MMA8451Q.
        8th column is the acceleration data in the Y axis measured by the sensor MMA8451Q.
        9th column is the acceleration data in the Z axis measured by the sensor MMA8451Q.
        
        Data are in bits with the following characteristics:

        ADXL345:
        Resolution: 13 bits
        Range: +-16g

        ITG3200
        Resolution: 16 bits
        Range: +-2000°/s

        MMA8451Q:
        Resolution: 14 bits
        Range: +-8g
'''

'\n        Each file contains nine columns and a different number of rows depending on the test length.\n        \n        1st column is the acceleration data in the X axis measured by the sensor ADXL345.\n        2nd column is the acceleration data in the Y axis measured by the sensor ADXL345.\n        3rd column is the acceleration data in the Z axis measured by the sensor ADXL345.\n\n        4th column is the rotation data in the X axis measured by the sensor ITG3200.\n        5th column is the rotation data in the Y axis measured by the sensor ITG3200.\n        6th column is the rotation data in the Z axis measured by the sensor ITG3200.\n\n        7th column is the acceleration data in the X axis measured by the sensor MMA8451Q.\n        8th column is the acceleration data in the Y axis measured by the sensor MMA8451Q.\n        9th column is the acceleration data in the Z axis measured by the sensor MMA8451Q.\n        \n        Data are in bits with the following characteristics:\

In [4]:
datapath = "./data/SisFall.csv"

df = pd.read_csv(datapath, index_col=0)
df.head(10)

,ADXL_x,ADXL_y,ADXL_z,ITG_x,ITG_y,ITG_z,MMA_x,MMA_y,MMA_z,subject,activity,trial
NaN,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,SA01,D01,R01
0.0,17.0,-179.0,-99.0,-18.0,-504.0,-352.0,76.0,-697.0,-279.0,SA01,D01,R01
1.0,15.0,-174.0,-90.0,-53.0,-568.0,-306.0,48.0,-675.0,-254.0,SA01,D01,R01
2.0,1.0,-176.0,-81.0,-84.0,-613.0,-271.0,-2.0,-668.0,-221.0,SA01,D01,R01
3.0,-10.0,-180.0,-77.0,-104.0,-647.0,-227.0,-34.0,-697.0,-175.0,SA01,D01,R01
4.0,-21.0,-191.0,-63.0,-128.0,-675.0,-191.0,-74.0,-741.0,-133.0,SA01,D01,R01
5.0,-37.0,-225.0,-59.0,-146.0,-700.0,-159.0,-110.0,-840.0,-103.0,SA01,D01,R01
6.0,-36.0,-243.0,-46.0,-166.0,-722.0,-131.0,-147.0,-948.0,-75.0,SA01,D01,R01
7.0,-44.0,-271.0,-38.0,-190.0,-738.0,-107.0,-171.0,-1073.0,-38.0,SA01,D01,R01
8.0,-51.0,-312.0,-33.0,-210.0,-752.0,-90.0,-200.0,-1191.0,3.0,SA01,D01,R01


In [5]:
print (df.columns)
print (set(df['subject']))
features = df.copy()

dnum = 200
fnum = 365

Index(['ADXL_x', 'ADXL_y', 'ADXL_z', 'ITG_x', 'ITG_y', 'ITG_z', 'MMA_x',
       'MMA_y', 'MMA_z', 'subject', 'activity', 'trial'],
      dtype='object')
{'SA02', 'SE11', 'SA05', 'SA11', 'SA22', 'SE14', 'SA20', 'SE12', 'SA08', 'SE03', 'SE08', 'SE09', 'SA07', 'SA14', 'SE01', 'SA10', 'SA23', 'SE06', 'SE04', 'SE15', 'SA01', 'SE05', 'SE07', 'SA16', 'SE13', 'SA09', 'SA18', 'SA12', 'SE02', 'SA13', 'SA21', 'SA15', 'SA17', 'SA04', 'SA06', 'SE10', 'SA19', 'SA03'}


In [6]:
D01 = features[features['activity']=='D01'].head(dnum).copy()
D02 = features[features['activity']=='D02'].head(dnum).copy()
D03 = features[features['activity']=='D03'].head(dnum).copy()
D04 = features[features['activity']=='D04'].head(dnum).copy()
D05 = features[features['activity']=='D05'].head(dnum).copy()
D06 = features[features['activity']=='D06'].head(dnum).copy()
D07 = features[features['activity']=='D07'].head(dnum).copy()
D08 = features[features['activity']=='D08'].head(dnum).copy()
D09 = features[features['activity']=='D09'].head(dnum).copy()
D10 = features[features['activity']=='D10'].head(dnum).copy()
D11 = features[features['activity']=='D11'].head(dnum).copy()
D12 = features[features['activity']=='D12'].head(dnum).copy()
D13 = features[features['activity']=='D13'].head(dnum).copy()
D14 = features[features['activity']=='D14'].head(dnum).copy()
D15 = features[features['activity']=='D15'].head(dnum).copy()
D16 = features[features['activity']=='D16'].head(dnum).copy()
D17 = features[features['activity']=='D17'].head(dnum).copy()
D18 = features[features['activity']=='D18'].head(dnum).copy()
D19 = features[features['activity']=='D19'].head(dnum).copy()

In [7]:
F01 = features[features['activity']=='F01'].head(fnum).copy()
F02 = features[features['activity']=='F02'].head(fnum).copy()
F03 = features[features['activity']=='F03'].head(fnum).copy()
F04 = features[features['activity']=='F04'].head(fnum).copy()
F05 = features[features['activity']=='F05'].head(fnum).copy()
F06 = features[features['activity']=='F06'].head(fnum).copy()
F07 = features[features['activity']=='F07'].head(fnum).copy()
F08 = features[features['activity']=='F08'].head(fnum).copy()
F09 = features[features['activity']=='F09'].head(fnum).copy()
F10 = features[features['activity']=='F10'].head(fnum).copy()
F11 = features[features['activity']=='F11'].head(fnum).copy()
F12 = features[features['activity']=='F12'].head(fnum).copy()
F13 = features[features['activity']=='F13'].head(fnum).copy()
F14 = features[features['activity']=='F14'].head(fnum).copy()
F15 = features[features['activity']=='F15'].head(fnum).copy()

In [8]:
df = pd.DataFrame()
df = pd.concat([df, F01, F02, F03, F04, F05, F06, F07, F08, F09, F10, F11, F12, F13, F14, F15])
df = pd.concat([df, D01, D02, D03, D04, D05, D06, D07, D08, D09, D10, D11, D15, D16, D17, D18, D19])

In [9]:
df['activity'].value_counts()

activity
F01    365
F09    365
F02    365
F15    365
F14    365
F13    365
F11    365
F10    365
F12    365
F08    365
F07    365
F06    365
F05    365
F04    365
F03    365
D09    200
D18    200
D17    200
D16    200
D15    200
D11    200
D10    200
D01    200
D08    200
D07    200
D06    200
D05    200
D04    200
D03    200
D02    200
D19    200
Name: count, dtype: int64

In [10]:
features = df.copy()

In [11]:
features = features.replace({'activity':'D01'},0)
features = features.replace({'activity':'D01'},0)
features = features.replace({'activity':'D02'},0)
features = features.replace({'activity':'D03'},0)
features = features.replace({'activity':'D04'},0)
features = features.replace({'activity':'D05'},0)
features = features.replace({'activity':'D06'},0)
features = features.replace({'activity':'D07'},0)
features = features.replace({'activity':'D08'},0)
features = features.replace({'activity':'D09'},0)
features = features.replace({'activity':'D10'},0)
features = features.replace({'activity':'D11'},0)
features = features.replace({'activity':'D12'},0)
features = features.replace({'activity':'D13'},0)
features = features.replace({'activity':'D14'},0)
features = features.replace({'activity':'D15'},0)
features = features.replace({'activity':'D16'},0)
features = features.replace({'activity':'D17'},0)
features = features.replace({'activity':'D18'},0)
features = features.replace({'activity':'D19'},0)
features = features.replace({'activity':'F01'},1)
features = features.replace({'activity':'F02'},1)
features = features.replace({'activity':'F03'},1)
features = features.replace({'activity':'F04'},1)
features = features.replace({'activity':'F05'},1)
features = features.replace({'activity':'F06'},1)
features = features.replace({'activity':'F07'},1)
features = features.replace({'activity':'F08'},1)
features = features.replace({'activity':'F09'},1)
features = features.replace({'activity':'F10'},1)
features = features.replace({'activity':'F11'},1)
features = features.replace({'activity':'F12'},1)
features = features.replace({'activity':'F13'},1)
features = features.replace({'activity':'F14'},1)
features = features.replace({'activity':'F15'},1)
features = features.replace({'activity':'F15'},1)

/var/folders/_z/gryfr07n59jgb3wrd062h1ym0000gn/T/ipykernel_82671/4220643108.py:35: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  features = features.replace({'activity':'F15'},1)


In [12]:
features['activity'].value_counts()

activity
1    5475
0    3200
Name: count, dtype: int64

In [13]:
features

,ADXL_x,ADXL_y,ADXL_z,ITG_x,ITG_y,ITG_z,MMA_x,MMA_y,MMA_z,subject,activity,trial
NaN,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,SA01,1,R01
0.0,-9.0,-257.0,-25.0,84.0,247.0,27.0,-120.0,-987.0,63.0,SA01,1,R01
1.0,-3.0,-263.0,-23.0,99.0,258.0,35.0,-110.0,-1016.0,68.0,SA01,1,R01
2.0,-1.0,-270.0,-22.0,114.0,272.0,45.0,-94.0,-1037.0,69.0,SA01,1,R01
3.0,1.0,-277.0,-24.0,127.0,286.0,57.0,-81.0,-1062.0,69.0,SA01,1,R01
...,...,...,...,...,...,...,...,...,...,...,...,...
194.0,4.0,-259.0,-21.0,-56.0,2.0,-5.0,18.0,-1008.0,35.0,SA01,0,R01
195.0,9.0,-261.0,-27.0,-59.0,2.0,-5.0,23.0,-1014.0,35.0,SA01,0,R01
196.0,4.0,-261.0,-25.0,-62.0,3.0,-5.0,20.0,-1011.0,33.0,SA01,0,R01
197.0,4.0,-262.0,-21.0,-65.0,3.0,-4.0,18.0,-1009.0,36.0,SA01,0,R01


In [14]:
print (set(features['subject']))

{'SA01'}


In [15]:
features = features[[ft for ft in features.columns if ft not in ['subject', 'trial']]]
print(features.columns)

Index(['ADXL_x', 'ADXL_y', 'ADXL_z', 'ITG_x', 'ITG_y', 'ITG_z', 'MMA_x',
       'MMA_y', 'MMA_z', 'activity'],
      dtype='object')


In [16]:
from sklearn.preprocessing import StandardScaler

feature_columns = [col for col in features.columns if col != 'activity']

scaler = StandardScaler()

features_scaled = features.copy()
features_scaled[feature_columns] = scaler.fit_transform(features[feature_columns])

In [17]:
adl_count = features_scaled['activity'].value_counts()[0]
fall_count = features_scaled['activity'].value_counts()[1]

adl_mask = features_scaled['activity'] == 0
adl_data = features_scaled[adl_mask]
adl1 = adl_data.iloc[:int(adl_count * 0.6)]
adl2 = adl_data.iloc[int(adl_count * 0.6):]

fall_mask = features_scaled['activity'] == 1
fall_data = features_scaled[fall_mask]
fall1 = fall_data.iloc[:int(fall_count * 0.6)]
fall2 = fall_data.iloc[int(fall_count * 0.6):]

print(adl1.shape, adl2.shape, fall1.shape, fall2.shape)

(1920, 10) (1280, 10) (3285, 10) (2190, 10)


In [18]:
def split_sequences(sequences, n_steps):
    X, y = [], []
    for i in range(len(sequences) - n_steps + 1):
        end_ix = i + n_steps
        
        seq_x = sequences.iloc[i:end_ix].iloc[:, :-1] 
        seq_y = sequences.iloc[end_ix-1].iloc[-1]     
        
        X.append(seq_x.values)  
        y.append(seq_y)
    
    return np.array(X), np.array(y)

In [19]:
WINDOW_LENGTH = 20
train_data, train_labels = split_sequences(adl1, WINDOW_LENGTH)
test_data, test_labels = split_sequences(pd.concat([adl2, fall2]), WINDOW_LENGTH)
print (f"after : {train_data.shape}, {test_data.shape}")
print (train_labels.shape, test_labels.shape)

after : (1901, 20, 9), (3451, 20, 9)
(1901,) (3451,)


----

In [20]:
BATCH_SIZE = 32
EPOCHS = 200
LEARNING_RATE = 0.001
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [21]:
class SisFallDataset(Dataset):
    def __init__(self, data):
        if isinstance(data, torch.Tensor):
            self.data = data.float()  
        else:
            self.data = torch.FloatTensor(data) 
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]
    
train_dataset = SisFallDataset(train_data)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [22]:
class DoubleConv(nn.Module): # Enhanced version
    def __init__(self, in_channels, out_channels, time_embed_dim=32):
        super().__init__()
        
        self.time_mlp = nn.Sequential(
            nn.Linear(time_embed_dim, out_channels),
            nn.ReLU(),
            nn.Linear(out_channels, out_channels)
        )
        
        self.conv = nn.Sequential(
            nn.Conv1d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm1d(out_channels),
            nn.GELU(),  
            nn.Dropout(0.2),
            nn.Conv1d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm1d(out_channels),
            nn.GELU(),
            nn.Dropout(0.2)
        )
        
        self.residual = nn.Conv1d(in_channels, out_channels, kernel_size=1) if in_channels != out_channels else nn.Identity()
    
    def forward(self, x, t=None):
        if t is not None:
            time_embed = self.time_mlp(t)
            x = x + time_embed.unsqueeze(-1)
        
        conv_out = self.conv(x)
        return conv_out + self.residual(x)
    

class UNet1D(nn.Module):
    def __init__(self, in_channels=9, out_channels=9):
        super(UNet1D, self).__init__()
        
        # Encoder
        self.enc1 = DoubleConv(in_channels, 64)
        self.pool1 = nn.MaxPool1d(kernel_size=2, stride=2)
        
        self.enc2 = DoubleConv(64, 128)
        self.pool2 = nn.MaxPool1d(kernel_size=2, stride=2)
        
        self.enc3 = DoubleConv(128, 256)
        self.pool3 = nn.MaxPool1d(kernel_size=2, stride=2)
        
        # Bottleneck
        self.bottleneck = DoubleConv(256, 512)
        
        # Decoder
        self.upconv3 = nn.ConvTranspose1d(512, 256, kernel_size=2, stride=2, padding=0)
        self.dec3 = DoubleConv(512, 256)  # 512 = 256(upconv) + 256(skip)
        
        self.upconv2 = nn.ConvTranspose1d(256, 128, kernel_size=2, stride=2, padding=0)
        self.dec2 = DoubleConv(256, 128)  # 256 = 128(upconv) + 128(skip)
        
        self.upconv1 = nn.ConvTranspose1d(128, 64, kernel_size=2, stride=2, padding=0)
        self.dec1 = DoubleConv(128, 64)   # 128 = 64(upconv) + 64(skip)
        
        self.final_conv = nn.Conv1d(64, out_channels, kernel_size=1)
        
    def calculate_padding(self, x_size, target_size):
        """Helper function to calculate padding needed"""
        diff = target_size - x_size
        return diff // 2, diff - diff // 2

    def forward(self, x, t):
        # 입력이 튜플일 경우 처리
        if isinstance(x, tuple):
            x = x[0]

        # 데이터 차원 확인 및 조정
        if len(x.shape) == 2:
            x = x.unsqueeze(0)  # 배치 차원 추가

        # 차원 순서 변경
        x = x.permute(0, 2, 1) if x.shape[1] != x.shape[2] else x

        e1 = self.enc1(x)           # Save for skip connection
        p1 = self.pool1(e1)

        e2 = self.enc2(p1)          # Save for skip connection
        p2 = self.pool2(e2)

        e3 = self.enc3(p2)          # Save for skip connection
        p3 = self.pool3(e3)

        b = self.bottleneck(p3)

        d3 = self.upconv3(b)
        if d3.shape[-1] != e3.shape[-1]:
            pad_left, pad_right = self.calculate_padding(d3.shape[-1], e3.shape[-1])
            d3 = F.pad(d3, (pad_left, pad_right))
        d3 = torch.cat([d3, e3], dim=1)
        d3 = self.dec3(d3)

        d2 = self.upconv2(d3)
        if d2.shape[-1] != e2.shape[-1]:
            pad_left, pad_right = self.calculate_padding(d2.shape[-1], e2.shape[-1])
            d2 = F.pad(d2, (pad_left, pad_right))
        d2 = torch.cat([d2, e2], dim=1)
        d2 = self.dec2(d2)

        d1 = self.upconv1(d2)
        if d1.shape[-1] != e1.shape[-1]:
            pad_left, pad_right = self.calculate_padding(d1.shape[-1], e1.shape[-1])
            d1 = F.pad(d1, (pad_left, pad_right))
        d1 = torch.cat([d1, e1], dim=1)
        d1 = self.dec1(d1)

        out = self.final_conv(d1)
        out = out.permute(0, 2, 1)

        return out

In [23]:
def cosine_beta_schedule(timesteps, s=0.008):
    steps = timesteps + 1
    x = torch.linspace(0, timesteps, steps)
    alphas_cumprod = torch.cos(((x / timesteps) + s) / (1 + s) * torch.pi * 0.5) ** 2
    alphas_cumprod = alphas_cumprod / alphas_cumprod[0]
    betas = 1 - (alphas_cumprod[1:] / alphas_cumprod[:-1])
    return torch.clip(betas, 0.0001, 0.02)

T = 500  
beta = cosine_beta_schedule(T)
alpha = 1 - beta
alpha_bar = torch.cumprod(alpha, dim=0)

def q_sample(x_start, t):
    noise = torch.randn_like(x_start)
    sqrt_alpha_bar = torch.sqrt(alpha_bar[t])[:, None, None]  
    sqrt_one_minus_alpha_bar = torch.sqrt(1 - alpha_bar[t])[:, None, None] 
    return sqrt_alpha_bar * x_start + sqrt_one_minus_alpha_bar * noise, noise  

def p_sample(model, x_t, t):
    predicted_noise = model(x_t, t)
    sqrt_alpha = torch.sqrt(alpha_bar[t])[:, None, None]
    sqrt_one_minus_alpha = torch.sqrt(1 - alpha_bar[t])[:, None, None]
    return (x_t - sqrt_one_minus_alpha * predicted_noise) / sqrt_alpha

In [24]:
model = UNet1D()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [25]:
def train_model(model, train_loader, criterion, optimizer, epochs=EPOCHS):
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        
        for batch_data in train_loader:
            # 데이터가 튜플일 경우 첫 번째 원소 사용
            if isinstance(batch_data, tuple):
                batch_data = batch_data[0]
            
            optimizer.zero_grad()
            
            # 시간 단계 선택
            batch_size = batch_data.size(0)
            t = torch.randint(0, T, (batch_size,))
            
            # 노이즈 추가
            noisy_data = q_sample(batch_data, t)
            true_noise = torch.randn_like(batch_data)
            
            # 모델 예측
            predicted_noise = model(noisy_data, t)
            
            # 손실 계산
            loss = criterion(predicted_noise, true_noise)
            
            # 그래디언트 클리핑
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
        avg_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch+1}, Loss: {avg_loss:.4f}")
        
train_model(model, train_loader, criterion, optimizer)

Epoch 1, Loss: 1.0507
Epoch 2, Loss: 1.0091
Epoch 3, Loss: 1.0054
Epoch 4, Loss: 1.0019
Epoch 5, Loss: 0.9981
Epoch 6, Loss: 1.0057
Epoch 7, Loss: 1.0035
Epoch 8, Loss: 1.0024
Epoch 9, Loss: 1.0011
Epoch 10, Loss: 1.0011
Epoch 11, Loss: 1.0034
Epoch 12, Loss: 1.0009
Epoch 13, Loss: 1.0000
Epoch 14, Loss: 1.0023
Epoch 15, Loss: 0.9957
Epoch 16, Loss: 0.9984
Epoch 17, Loss: 1.0005
Epoch 18, Loss: 0.9971
Epoch 19, Loss: 1.0004
Epoch 20, Loss: 1.0011
Epoch 21, Loss: 1.0028
Epoch 22, Loss: 1.0004
Epoch 23, Loss: 0.9983
Epoch 24, Loss: 0.9993
Epoch 25, Loss: 1.0012
Epoch 26, Loss: 0.9997
Epoch 27, Loss: 0.9990
Epoch 28, Loss: 0.9988
Epoch 29, Loss: 1.0014
Epoch 30, Loss: 1.0008
Epoch 31, Loss: 1.0029
Epoch 32, Loss: 1.0027
Epoch 33, Loss: 0.9995
Epoch 34, Loss: 1.0018
Epoch 35, Loss: 1.0021
Epoch 36, Loss: 1.0048
Epoch 37, Loss: 0.9988
Epoch 38, Loss: 1.0011
Epoch 39, Loss: 1.0030
Epoch 40, Loss: 0.9989
Epoch 41, Loss: 1.0005
Epoch 42, Loss: 1.0045
Epoch 43, Loss: 0.9931
Epoch 44, Loss: 0.99

In [26]:
def advanced_anomaly_detection(model, test_loader, train_data):
    model.eval()
    
    # 훈련 데이터의 재구성 오차 분포 분석
    train_reconstructions = []
    with torch.no_grad():
        for batch in train_loader:
            t = torch.randint(0, T, (batch.size(0),))
            noisy_data = q_sample(batch, t)
            recon = model(noisy_data, t)
            train_reconstructions.append(
                torch.mean((recon - batch)**2, dim=(1,2)).numpy()
            )
    
    train_reconstructions = np.concatenate(train_reconstructions)
    
    # 재구성 오차의 통계적 특성
    mu = np.mean(train_reconstructions)
    sigma = np.std(train_reconstructions)
    
    # 고급 이상 탐지
    anomalies = []
    anomaly_scores = []
    
    with torch.no_grad():
        for batch_data in test_loader:
            t = torch.randint(0, T, (batch_data.size(0),))
            noisy_data = q_sample(batch_data, t)
            recon = model(noisy_data, t)
            
            # 다중 기준 이상 탐지
            reconstruction_errors = torch.mean((recon - batch_data)**2, dim=(1,2)).numpy()
            
            for i, error in enumerate(reconstruction_errors):
                # Z-score 기반 이상 탐지
                z_score = (error - mu) / sigma
                
                # 쿼드런트 방법
                if error > mu + 2*sigma and z_score > 2:
                    anomalies.append(i)
                    anomaly_scores.append(error)
    
    return anomalies, anomaly_scores

In [27]:
test_tensor = torch.tensor(test_data)
test_loader = DataLoader(SisFallDataset(test_tensor), batch_size=32, shuffle=False)

anomalies, error_values = detect_anomalies(model, test_loader)

error_df = pd.DataFrame({
    'reconstruction_error': error_values,
    'true_class': test_labels
})

error_df

NameError: name 'detect_anomalies' is not defined

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(x='true_class', y='reconstruction_error', data=error_df, showfliers=False)
plt.ylabel('Distribution')
plt.xticks(rotation=90)
plt.show()

In [ ]:
plt.figure(figsize=(10, 4))
plt.title('MSE Distribution')
plt.hist(error_values[test_labels==0], bins=50, alpha=0.5, color='blue', label='Normal(ADL)', density=True)
plt.hist(error_values[test_labels==1], bins=50, alpha=0.5, color='red', label='Abnormal(Fall)', density=True)
plt.xlabel('MSE')
plt.ylabel('Density')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
threshold = np.percentile(error_values, 95) 

y_pred = (error_values > threshold).astype(int)
print("\nClassification Report:")
print(classification_report(test_labels, y_pred))

In [ ]:
groups = error_df.groupby('true_class')
fig, ax = plt.subplots()

for name, group in groups:
    ax.plot(group.index, group.reconstruction_error, marker='^', ms=3.5, linestyle='',
            label="Normal" if name == 0 else "Fall")
ax.hlines(threshold, ax.get_xlim()[0], ax.get_xlim()[1], colors="r", zorder=100, label='Threshold')
ax.legend()
plt.title("Reconstruction error for different classes")
plt.ylabel("Reconstruction error")
plt.xlabel("Data point index")
plt.show()

In [ ]:
LABELS = ["ADL", "FALL"]
plt.figure(figsize=(7, 7))
conf_matrix = confusion_matrix(test_labels, y_pred)
sns.heatmap(conf_matrix, xticklabels=LABELS, yticklabels=LABELS, annot=True, fmt="d")
plt.title("Confusion matrix")
plt.ylabel('True class')
plt.xlabel('Predicted class')
plt.show()
